## Probability Distribution Tables Representation and Inference

**COMP9418-19T3, W02 Tutorial**

- Instructor: Gustavo Batista
- School of Computer Science and Engineering, UNSW Sydney
- Notebook designed by Gustavo Batista and Jeremy Gillen
- Last update 18th August 2021

In this week's tutorial, we will design a data structure for probability table representation and implement four operations over this representation. This code will be used in the next tutorials to perform inference and learning over graphical models.

## Technical prerequisites

We will use Jupyter Notebooks in the practical part of the tutorials. There are three main ways to run these notebooks:

1. *Google Colab*. The links available in WebCMS will open the course notebooks in [Google Colab](https://colab.research.google.com/). Google Colab has all the packages necessary to run these notebooks, so no installation is required. **This is our prefered approach**.

2. *CSE VLAB*. We have installed the necessary packages in the CSE computers. You have to copy the notebook to your CSE account in VLAB to execute them. There are two possibilities to access VLAB. The most recommended is installing a [VNC software](https://taggi.cse.unsw.edu.au/FAQ/Really_quick_guide_to_VLAB/) in your computer. Alternatively, you can access these computers through a web interface using [CSE VLAB Gateway](https://vlabgateway.cse.unsw.edu.au/).

3. *Your computer*. If you do not have Jupyter installed in your computer, we recommend installing [Anaconda](https://www.anaconda.com/distribution). Anaconda conveniently installs Python, the Jupyter Notebook, and other commonly used packages for scientific computing and data science. To render a graphical visualization of some graphs in this notebook, you also need to [install Graphviz](http://www.graphviz.org/download). If you have conda installed in your computer, you can try the command ```conda install python-graphviz``` directly. From our experience, Graphviz is a little troublesome to install in some systems. For instance, ```conda install python-graphviz``` often does not work on Linux systems and we did not have any success using ```pip3 install graphviz``` on most systems, so do *not* use ```pip3```.
You will need specific packages installed to run this notebook.
We will use the [tabulate](https://pypi.org/project/tabulate/) library to print probability tables for debugging. If you don't have it installed, use the command ```pip install tabulate```.

Let's import some useful modules for later use.

In [ ]:
# combinatorics
from itertools import product
# table formating for screen output
from tabulate import tabulate
# numpy is used for n-dimensional arrays
import numpy as np
import copy

## Representing probability tables

We will represent the distributions of discrete variables using probability tables. For example, here are 3 random variables, $X$, $Y$, and $Z$, each on $\{0,1\}$.

  | X | Y | Z | p(X,Y,Z) |
  |---|---|---|----------|
  | 0 | 0 | 0 | 0 | 
  | 0 | 0 | 1 | 1/12 | 
  | 0 | 1 | 0 | 1/12 | 
  | 0 | 1 | 1 | 1/6 | 
  | 1 | 0 | 0 | 1/12 | 
  | 1 | 0 | 1 | 1/6 | 
  | 1 | 1 | 0 | 1/6 | 
  | 1 | 1 | 1 | 1/4 | 

Another example is a table that represents a conditional distribution for, say, $p(Z|X,Y)$.

  | X | Y | Z | p(Z &#124; X,Y)        |
  |---|---|---|---------------------------|
  | 0 | 0 | 0 | 0 | 
  | 0 | 0 | 1 | 1 | 
  | 0 | 1 | 0 | 1/3 | 
  | 0 | 1 | 1 | 2/3 | 
  | 1 | 0 | 0 | 1/3 | 
  | 1 | 0 | 1 | 2/3 | 
  | 1 | 1 | 0 | 2/5 | 
  | 1 | 1 | 1 | 3/5 | 

We will use the term **factor** to denote a probability table, joint or conditional. 

The natural question is how we represent tables like these in Python. We should note that to define a factor table completely, we need to specify three pieces of information:

1. The domain of the factor, i.e., which variables belong to the factor;

2. The outcome space of each variable, i.e, the possible values of each random variable.

3. The probabilities associated with each possible combination of variables values in the factor domain.

We can create a python class to store this information, and create methods for getting and setting individual probabilities.

We will use the methods `__getitem__` and `__setitem__` to access and set probabilities in this class. You can see more about these special methods [here](https://docs.python.org/3/reference/datamodel.html#emulating-container-types). These will allow us to use square brackets to access the probabilities of the factor, similar to how you access elements of a list or numpy array.

In [ ]:
class Factor:
    '''
    Factors are a generalisation of discrete probability distributions over one or more random variables.
    Each variable must have a name (which may be a string or integer).
    The domain of the factor specifies which variables the factor operates over.
    The outcomeSpace specifies the possible values of each random variable. 
    
    
    The probabilities are stored in a n-dimensional numpy array, using the domain and outcomeSpace
    as dimension and row labels respectively.
    '''
    def __init__(self, domain, outcomeSpace, table=None):
        '''
        Inititalise a factor with a given domain and outcomeSpace. 
        All probabilities are set to zero by default.
        '''
        self.domain = tuple(domain) # tuple of variable names, each of which may be a string or integer.
        
        # Initialise the outcomeSpace, which is provided as a dictionary.
        self.outcomeSpace = copy.copy(outcomeSpace)

        if table is None:
            # By default, intitialize with a uniform distribution
            self.table = np.ones(shape=tuple(len(outcomeSpace[var]) for var in self.domain))
            self.table = self.table/np.sum(self.table)
        else:
            self.table = table        
    
    def __getitem__(self, outcomes):
        '''
        This function allows direct access to individual probabilities.
        E.g. if the factor represents a joint distribution over variables 'A','B','C','D', each with outcomeSpace [0,1,2],
        then `factor[0,1,0,2]` will return the probability that the four variables are set to 0,1,0,2 respectively.
        
        The outcomes used for indexing may also be strings. If in the example above the outcomeSpace of each variable is
        ['true','false'], then `factor['true','false','true','true']` will return the probability that 
        A='true', B='false', C='true' and D='true'. 
        
        The order of the indicies is determined by the order of the variables in self.domain.
        '''
        
        # check if only a single index was used.
        if not isinstance(outcomes, tuple):
            outcomes = (outcomes,)
            
        assert(len(outcomes) == len(self.domain)) # confirm there is a correct number of indicies
        
        # convert outcomes into array indicies
        indicies = tuple(self.outcomeSpace[var].index(outcomes[i]) for i, var in enumerate(self.domain))
        return self.table[indicies]
    
    def __setitem__(self, outcomes, new_value):
        '''
        This function is called when setting a probability. E.g. `factor[0,1,0,2] = 0.5`.        
        '''
        if not isinstance(outcomes, tuple):
            outcomes = (outcomes,)
        indicies = tuple(self.outcomeSpace[var].index(outcomes[i]) for i, var in enumerate(self.domain))
        self.table[indicies] = new_value
        
    def __str__(self):
        pass
        
    def join(self, other):
        pass
    
    def evidence(self, **kwargs):
        pass

    def marginalize(self, var):
        pass

    def normalize(self):
        pass
    
    def copy(self):
        return copy.deepcopy(self)

For instance, we can define 3 random variables ($S$, $T$ and $W$) with the following possible outcomes.

In [ ]:
outcomeSpace = {
                'S':('summer','winter'),
                'T':('hot','cold'),
                'W':('sun','rain'),
               }

Now we can represent the table:

| S      | P(S)   |
|:------:|:------:|
| summer | 0.5    |
| winter | 0.5    |

with:

In [ ]:
s_prob = Factor(('S',), outcomeSpace)
s_prob['summer'] = 0.5
s_prob['winter'] = 0.5

Likewise, the table

|   S    | T    | P(T&#124;S)    |
|:------:|:----:|:---------:|
| summer |  hot | 0.7       |
| summer | cold | 0.3       |
| winter | hot  | 0.3       |
| winter | cold | 0.7       |

is represented as:

In [ ]:
t_prob = Factor(('S','T'), outcomeSpace)
t_prob['summer','hot']  = 0.7
t_prob['summer','cold'] = 0.3
t_prob['winter','hot']  = 0.3
t_prob['winter','cold'] = 0.7

### Exercise

It is your turn, represent the table:

|  S     |  T   |  W   | P(W&#124;S,T)   |
|:------:|:----:|:----:|:----------:|
| summer |  hot |  sun |   0.86     |
| summer |  hot | rain |   0.14     |
| summer | cold |  sun |   0.67     |
| summer | cold | rain |   0.33     |
| winter |  hot |  sun |   0.67     |
| winter |  hot | rain |   0.33     |
| winter | cold |  sun |   0.43     |
| winter | cold | rain |   0.57     |


In [ ]:
w_prob = ...
...
...

In [ ]:
#Answer

w_prob = Factor(('S','T','W'), outcomeSpace)
w_prob['summer','hot','sun']   = 0.86
w_prob['summer','hot','rain']  = 0.14
w_prob['summer','cold','sun']  = 0.67
w_prob['summer','cold','rain'] = 0.33
w_prob['winter','hot','sun']   = 0.67
w_prob['winter','hot','rain']  = 0.33
w_prob['winter','cold','sun']  = 0.43
w_prob['winter','cold','rain'] = 0.57

Notice that the variables domain do not need to be restricted to two values. We can specify larger domains with more values. Often integers will be used to reduce the typing needed.

We need to implement four basic operations over factors:
    
1. Factor join: given two factors $f_1$ and $f_2$ with at least one variable in common, join these factors, creating a new factor $f$. The domain of the new factor has all variables in $dom(f_1) \cup dom(f_2)$.

2. Factor marginalization: given a factor $f$ eliminate one variable $v \in dom(f)$ by summing over all values of $v$.

3. Evidence observation: given a variable $X$ and a value $x$, set the evidence $X=x$. This means that the variable $X$ has been observed as having the value $x$. Consequently, the join and marginalization operations will restrict themselves to $x$ and ignore the remaining values of $X$.

4. Factor normalization: normalize the entries in a given factor so that all entries sum up to one.

Before we start, let's define a function to print out factors nicely to the screen. This function will help us to debug our code. For this task, we will use the [tabulate library](https://pypi.org/project/tabulate/) and the [product iterator provided by itertools](https://docs.python.org/2/library/itertools.html).

We will use the special method `__str__` to write this function, which will make it easy to print out a factor `f` using `print(f)`. You can read the documentation for special methods like this [here](https://docs.python.org/3/reference/datamodel.html#special-method-names).

In [ ]:
def __str__(self):
    '''
    This function determines the string representation of an object.
    In this case, the function prints out a row for every possible instantiation 
    of the factor, along with the associated probability.
    This function will be called whenever you print out this object, i.e.
    a_prob = Factor(...)
    print(a_prob)
    '''
    table = []
    outcomeSpaces = [self.outcomeSpace[var] for var in self.domain]
    for key in product(*outcomeSpaces):
        row = list(key)
        row.append(self[key])
        table.append(row)
    header = list(self.domain) + ['Pr']
    return tabulate(table, headers=header, tablefmt='fancy_grid') + '\n'

# The following line is one way of defining a method outside of the class definition.
# Usually we would put this function inside the Factor class definition
Factor.__str__ = __str__ 

# Testing
print(w_prob)

╒════════╤══════╤══════╤══════╕
│ S      │ T    │ W    │   Pr │
╞════════╪══════╪══════╪══════╡
│ summer │ hot  │ sun  │ 0.86 │
├────────┼──────┼──────┼──────┤
│ summer │ hot  │ rain │ 0.14 │
├────────┼──────┼──────┼──────┤
│ summer │ cold │ sun  │ 0.67 │
├────────┼──────┼──────┼──────┤
│ summer │ cold │ rain │ 0.33 │
├────────┼──────┼──────┼──────┤
│ winter │ hot  │ sun  │ 0.67 │
├────────┼──────┼──────┼──────┤
│ winter │ hot  │ rain │ 0.33 │
├────────┼──────┼──────┼──────┤
│ winter │ cold │ sun  │ 0.43 │
├────────┼──────┼──────┼──────┤
│ winter │ cold │ rain │ 0.57 │
╘════════╧══════╧══════╧══════╛




### Exercise

Print out the probability in the `w_prob` table corresponding to the row "summer", "cold", "rain".

In [ ]:
... # TODO

Ellipsis

In [ ]:
#Answer

print(w_prob['summer','cold','rain'])

0.33


If you implemented the call correctly, you should see the following output:

```
0.33
```

## Observing Evidence

Observing a value $x$ for a variable $X$ means that every row in the table that doesn't contain $x$ should have probability 0. We can also implement this by removing all those rows from the table.

Implementing this function by removing all unnecessary rows is a little more efficient for some applications, but requires selecting the right portion of the n-dimensional numpy array using advanced numpy *slicing*. See [here](https://numpy.org/doc/stable/reference/arrays.indexing.html) for more information on slicing, or see the next cell for an example.


In [ ]:
# We have the following table of shape (2,3).
# We want to select the just the first column, which corresponds 
# to the first outcome of the second variable.
# The new shape will be (2,1)
table = np.array([[0.20,0.10,0.10],
                  [0.05,0.15,0.50]])
print(table)
print()

# The usual way to do this is:
print("First method:")
new_table = table[:,0:1]
print(new_table)

# Another (equivalent) way of doing it, which makes it easier to 
# generate a slice programmatically
print("Second method")
slice_tuple = (slice(None),slice(0,1))
new_table = table[slice_tuple]
print(new_table)

[[0.2  0.1  0.1 ]
 [0.05 0.15 0.5 ]]

First method:
[[0.2 ]
 [0.05]]
Second method
[[0.2 ]
 [0.05]]


### Exercise

It is your turn, use what you've learned about slicing to implement the evidence function.

In [ ]:
def evidence(self, **kwargs):
    '''
    Sets evidence by modifying the outcomeSpace
    This function must be used to set evidence on all factors before joining,
    because it removes the irrelevant values from the factor. 

    Usage: fac.evidence(A='true',B='false')
    This returns a factor which has set the variable 'A' to 'true' and 'B' to 'false'.
    '''
    f = self.copy()
    evidence_dict = kwargs
    for var, value in evidence_dict.items():
        if var in f.domain:

            # find the row index that corresponds to var = value (use f.outcomeSpace)
            ... # TODO
            
            # create a tuple of "slice" objects, all of them `slice(None)` except
            # for the one corresponding to the `var` axis.
            ... # TODO (for loop or list comprehension required)

            # use the tuple to select the required section of f.table
            f.table = ... # TODO

            # modify the outcomeSpace to correspond to the changes just made to self.table
            f.outcomeSpace[var] = (value,)
    return f

Factor.evidence = evidence

#####################################
# Test code
#####################################

print(w_prob.evidence(S='summer'))
print(w_prob.evidence(T='cold'))
print(w_prob.evidence(W='rain',T='hot'))

TypeError: 'ellipsis' object is not subscriptable

In [ ]:
# Answer

def evidence(self, **kwargs):
    '''
    Sets evidence by modifying the outcomeSpace
    This function must be used to set evidence on all factors before joining,
    because it removes the relevant variable from the factor. 

    Usage: fac.evidence(A='true',B='false')
    This returns a factor which has set the variable 'A' to 'true' and 'B' to 'false'.
    '''
    f = self.copy()
    evidence_dict = kwargs
    for var, value in evidence_dict.items():
        if var in f.domain:

            # find the row index that corresponds to var = value
            index = f.outcomeSpace[var].index(value)

            # find the `var` axis and select only the row that corresponds to `value`
            # on all other axes, select every row
            slice_tuple = tuple(slice(index,index+1) if v == var else slice(None) for v in f.domain)
            f.table = f.table[slice_tuple]

            # modify the outcomeSpace to correspond to the changes just made to self.table
            f.outcomeSpace[var] = (value,)
    return f

Factor.evidence = evidence

#####################################
# Test code
#####################################

print(w_prob.evidence(S='summer'))
print(w_prob.evidence(T='cold'))
print(w_prob.evidence(W='rain',T='hot'))

╒════════╤══════╤══════╤══════╕
│ S      │ T    │ W    │   Pr │
╞════════╪══════╪══════╪══════╡
│ summer │ hot  │ sun  │ 0.86 │
├────────┼──────┼──────┼──────┤
│ summer │ hot  │ rain │ 0.14 │
├────────┼──────┼──────┼──────┤
│ summer │ cold │ sun  │ 0.67 │
├────────┼──────┼──────┼──────┤
│ summer │ cold │ rain │ 0.33 │
╘════════╧══════╧══════╧══════╛

╒════════╤══════╤══════╤══════╕
│ S      │ T    │ W    │   Pr │
╞════════╪══════╪══════╪══════╡
│ summer │ cold │ sun  │ 0.67 │
├────────┼──────┼──────┼──────┤
│ summer │ cold │ rain │ 0.33 │
├────────┼──────┼──────┼──────┤
│ winter │ cold │ sun  │ 0.43 │
├────────┼──────┼──────┼──────┤
│ winter │ cold │ rain │ 0.57 │
╘════════╧══════╧══════╧══════╛

╒════════╤═════╤══════╤══════╕
│ S      │ T   │ W    │   Pr │
╞════════╪═════╪══════╪══════╡
│ summer │ hot │ rain │ 0.14 │
├────────┼─────┼──────┼──────┤
│ winter │ hot │ rain │ 0.33 │
╘════════╧═════╧══════╧══════╛



If you implemented your code correctly, you should see the following output:

```
╒════════╤══════╤══════╤══════╕
│ S      │ T    │ W    │   Pr │
╞════════╪══════╪══════╪══════╡
│ summer │ hot  │ sun  │ 0.86 │
├────────┼──────┼──────┼──────┤
│ summer │ hot  │ rain │ 0.14 │
├────────┼──────┼──────┼──────┤
│ summer │ cold │ sun  │ 0.67 │
├────────┼──────┼──────┼──────┤
│ summer │ cold │ rain │ 0.33 │
╘════════╧══════╧══════╧══════╛

╒════════╤══════╤══════╤══════╕
│ S      │ T    │ W    │   Pr │
╞════════╪══════╪══════╪══════╡
│ summer │ cold │ sun  │ 0.67 │
├────────┼──────┼──────┼──────┤
│ summer │ cold │ rain │ 0.33 │
├────────┼──────┼──────┼──────┤
│ winter │ cold │ sun  │ 0.43 │
├────────┼──────┼──────┼──────┤
│ winter │ cold │ rain │ 0.57 │
╘════════╧══════╧══════╧══════╛

╒════════╤═════╤══════╤══════╕
│ S      │ T   │ W    │   Pr │
╞════════╪═════╪══════╪══════╡
│ summer │ hot │ rain │ 0.14 │
├────────┼─────┼──────┼──────┤
│ winter │ hot │ rain │ 0.33 │
╘════════╧═════╧══════╧══════╛

```

## Factor Join Operation

The central operation of inference is the factor multiplication or join. This operation will collapse two factors into a single factor. This operation should carefully match the values of the variables to provide the correct output.

The following cell demonstrates how to join two probability tables representing factors $P(A)$ and $P(B|A)$ into $P(A,B)$. This code takes advantage of [numpy broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html) when multiplying tables of different shapes. Make sure you understand how broadcasting work is working in the example below before trying the question.

In [ ]:
print("P(A):")
A_table = np.array([0.3,0.1,0.6]) # shape (3) domain: [A]
print(A_table)

print("P(B|A):")
B_given_A_table = np.array([[0.2, 0.8],
                         [0.1, 0.9],
                         [0.1, 0.9]]) # shape (3,2), domain: [A,B]
print(B_given_A_table)

# first, reshape A so that it has the same number and order of dimensions
A_table = np.expand_dims(A_table,-1) # insert new axis in the last position
# A_table now has shape (3,1) and domain [A, B]

print("P(B,A):")
# We can multiply the two arrays elementwise
print(A_table*B_given_A_table)

P(A):
[0.3 0.1 0.6]
P(B|A):
[[0.2 0.8]
 [0.1 0.9]
 [0.1 0.9]]
P(B,A):
[[0.06 0.24]
 [0.01 0.09]
 [0.06 0.54]]


### Exercise

Let's implement the function join. We will provide most of the code for you. You will need to fill in a few gaps to complete the implementation.

In [ ]:
def join(self, other):
    '''
    This function multiplies two factors.
    '''
    # confirm that any shared variables have the same outcomeSpace
    for var in set(other.domain).intersection(set(self.domain)):
        if self.outcomeSpace[var] != other.outcomeSpace[var]:
            raise IndexError('Incompatible outcomeSpaces. Make sure you set the same evidence on all factors')

    # extend current domain with any new variables required
    new_dom = list(self.domain) + list(set(other.domain) - set(self.domain)) 

    self_t = self.table
    other_t = other.table
    
    # to prepare for multiplying arrays, we need to make sure both arrays have the correct number of axes
    # We will do this by adding axes to the end of the shape of each array.
    num_new_axes = len(set(other.domain) - set(self.domain))
    for i in range(num_new_axes):
        # add an axis to self_t. E.g. if shape is [3,5], new shape will be [3,5,1]
        ... # TODO 
    num_new_axes = len(set(self.domain) - set(other.domain))
    for i in range(num_new_axes):
        # add an axis to other_t. E.g. if shape is [3,5], new shape will be [3,5,1]
        ... # TODO 

    # And we need the new axes to be transposed to the correct location
    old_order = list(other.domain) + list(set(self.domain) - set(other.domain)) 
    new_order = []
    for v in new_dom:
        new_order.append(old_order.index(v))
    other_t = ... # TODO reorder the axes of other_t to new_order (look up np.transpose)

    # Now that the arrays are all set up, we can rely on numpy broadcasting to work out which numbers need to be multiplied.
    # https://numpy.org/doc/stable/user/basics.broadcasting.html
    new_table = ... #TODO multiply the two arrays

    # The final step is to create the new outcomeSpace
    new_outcomeSpace = self.outcomeSpace.copy()
    new_outcomeSpace.update(other.outcomeSpace)

    # create a new factor out of the the new domain, outcomeSpace and table
    # in the following line, `self.__class__` is the same as `Factor` (except it doesn't break things when subclassing)
    return self.__class__(tuple(new_dom), new_outcomeSpace, table=new_table)

Factor.join = join # add the method to the factor class

#####################################
# Test code
#####################################

# reorder the domain of t_prob to make sure it still works
t_prob = Factor(('T','S'), outcomeSpace)
t_prob['hot','summer']  = 0.7
t_prob['cold','summer'] = 0.3
t_prob['hot','winter']  = 0.3
t_prob['cold','winter'] = 0.7

print(s_prob.join(t_prob))
s_prob_summer = s_prob.evidence(S='summer')
t_prob_summer = t_prob.evidence(S='summer')
print(s_prob_summer.join(t_prob_summer))

# back to original order
t_prob = Factor(('S','T'), outcomeSpace)
t_prob['summer','hot']  = 0.7
t_prob['summer','cold'] = 0.3
t_prob['winter','hot']  = 0.3
t_prob['winter','cold'] = 0.7

print(t_prob.join(w_prob))
print(w_prob.join(s_prob))

TypeError: 'ellipsis' object is not subscriptable

In [ ]:
# Answer

def join(self, other):
    '''
    This function multiplies two factors.
    '''
    # confirm that any shared variables have the same outcomeSpace
    for var in set(other.domain).intersection(set(self.domain)):
        if self.outcomeSpace[var] != other.outcomeSpace[var]:
            raise IndexError('Incompatible outcomeSpaces. Make sure you set the same evidence on all factors')

    # extend current domain with any new variables required
    new_dom = list(self.domain) + list(set(other.domain) - set(self.domain)) 

    self_t = self.table
    other_t = other.table
    
    # to prepare for multiplying arrays, we need to make sure both arrays have the correct number of axes
    # We will do this by adding dimensions of size 1 to the end of the shape of each array.
    num_new_axes = len(set(other.domain) - set(self.domain))
    for i in range(num_new_axes):
        # add an axis to self_t. E.g. if shape is [3,5], new shape will be [3,5,1]
        self_t = np.expand_dims(self_t,-1) 
    num_new_axes = len(set(self.domain) - set(other.domain))
    for i in range(num_new_axes):
        # add an axis to other_t. E.g. if shape is [3,5], new shape will be [3,5,1]
        other_t = np.expand_dims(other_t,-1) 

    # And we need the new axes to be transposed to the correct location
    old_order = list(other.domain) + list(set(self.domain) - set(other.domain)) 
    new_order = []
    for v in new_dom:
        new_order.append(old_order.index(v))
    other_t = np.transpose(other_t, new_order)

    # Now that the arrays are all set up, we can rely on numpy broadcasting to work out which numbers need to be multiplied.
    # https://numpy.org/doc/stable/user/basics.broadcasting.html
    new_table = self_t * other_t

    # The final step is to create the new outcomeSpace
    new_outcomeSpace = self.outcomeSpace.copy()
    new_outcomeSpace.update(other.outcomeSpace)

    # in the following line, `self.__class__` is the same as `Factor` (except it doesn't break things when subclassing)
    return self.__class__(tuple(new_dom), new_outcomeSpace, table=new_table)

Factor.join = join # add the method to the factor class

#####################################
# Test code
#####################################

# reorder the domain of t_prob to make sure it still works
t_prob = Factor(('T','S'), outcomeSpace)
t_prob['hot','summer']  = 0.7
t_prob['cold','summer'] = 0.3
t_prob['hot','winter']  = 0.3
t_prob['cold','winter'] = 0.7

print(s_prob.join(t_prob))
s_prob_summer = s_prob.evidence(S='summer')
t_prob_summer = t_prob.evidence(S='summer')
print(s_prob_summer.join(t_prob_summer))

# back to original order
t_prob = Factor(('S','T'), outcomeSpace)
t_prob['summer','hot']  = 0.7
t_prob['summer','cold'] = 0.3
t_prob['winter','hot']  = 0.3
t_prob['winter','cold'] = 0.7

print(t_prob.join(w_prob))
print(w_prob.join(s_prob))

╒════════╤══════╤══════╕
│ S      │ T    │   Pr │
╞════════╪══════╪══════╡
│ summer │ hot  │ 0.35 │
├────────┼──────┼──────┤
│ summer │ cold │ 0.15 │
├────────┼──────┼──────┤
│ winter │ hot  │ 0.15 │
├────────┼──────┼──────┤
│ winter │ cold │ 0.35 │
╘════════╧══════╧══════╛

╒════════╤══════╤══════╕
│ S      │ T    │   Pr │
╞════════╪══════╪══════╡
│ summer │ hot  │ 0.35 │
├────────┼──────┼──────┤
│ summer │ cold │ 0.15 │
╘════════╧══════╧══════╛

╒════════╤══════╤══════╤═══════╕
│ S      │ T    │ W    │    Pr │
╞════════╪══════╪══════╪═══════╡
│ summer │ hot  │ sun  │ 0.602 │
├────────┼──────┼──────┼───────┤
│ summer │ hot  │ rain │ 0.098 │
├────────┼──────┼──────┼───────┤
│ summer │ cold │ sun  │ 0.201 │
├────────┼──────┼──────┼───────┤
│ summer │ cold │ rain │ 0.099 │
├────────┼──────┼──────┼───────┤
│ winter │ hot  │ sun  │ 0.201 │
├────────┼──────┼──────┼───────┤
│ winter │ hot  │ rain │ 0.099 │
├────────┼──────┼──────┼───────┤
│ winter │ cold │ sun  │ 0.301 │
├────────┼──────┼───

If you implemented the join operation correctly, you should see the following output:

```
╒════════╤══════╤══════╕
│ S      │ T    │   Pr │
╞════════╪══════╪══════╡
│ summer │ hot  │ 0.35 │
├────────┼──────┼──────┤
│ summer │ cold │ 0.15 │
├────────┼──────┼──────┤
│ winter │ hot  │ 0.15 │
├────────┼──────┼──────┤
│ winter │ cold │ 0.35 │
╘════════╧══════╧══════╛

╒════════╤══════╤══════╕
│ S      │ T    │   Pr │
╞════════╪══════╪══════╡
│ summer │ hot  │ 0.35 │
├────────┼──────┼──────┤
│ summer │ cold │ 0.15 │
╘════════╧══════╧══════╛

╒════════╤══════╤══════╤═══════╕
│ S      │ T    │ W    │    Pr │
╞════════╪══════╪══════╪═══════╡
│ summer │ hot  │ sun  │ 0.602 │
├────────┼──────┼──────┼───────┤
│ summer │ hot  │ rain │ 0.098 │
├────────┼──────┼──────┼───────┤
│ summer │ cold │ sun  │ 0.201 │
├────────┼──────┼──────┼───────┤
│ summer │ cold │ rain │ 0.099 │
├────────┼──────┼──────┼───────┤
│ winter │ hot  │ sun  │ 0.201 │
├────────┼──────┼──────┼───────┤
│ winter │ hot  │ rain │ 0.099 │
├────────┼──────┼──────┼───────┤
│ winter │ cold │ sun  │ 0.301 │
├────────┼──────┼──────┼───────┤
│ winter │ cold │ rain │ 0.399 │
╘════════╧══════╧══════╧═══════╛

╒════════╤══════╤══════╤═══════╕
│ S      │ T    │ W    │    Pr │
╞════════╪══════╪══════╪═══════╡
│ summer │ hot  │ sun  │ 0.43  │
├────────┼──────┼──────┼───────┤
│ summer │ hot  │ rain │ 0.07  │
├────────┼──────┼──────┼───────┤
│ summer │ cold │ sun  │ 0.335 │
├────────┼──────┼──────┼───────┤
│ summer │ cold │ rain │ 0.165 │
├────────┼──────┼──────┼───────┤
│ winter │ hot  │ sun  │ 0.335 │
├────────┼──────┼──────┼───────┤
│ winter │ hot  │ rain │ 0.165 │
├────────┼──────┼──────┼───────┤
│ winter │ cold │ sun  │ 0.215 │
├────────┼──────┼──────┼───────┤
│ winter │ cold │ rain │ 0.285 │
╘════════╧══════╧══════╧═══════╛

```

## Factor Marginalization Operation

Marginalization is the operation that eliminates a given variable $X$ from a factor $f$ by summing over all possible values of $X$ in $f$. The marginalize function will return a new factor $f'$ to avoid messing with existing factors. The new factor $f'$ will have the same domain as $f$, but with the elimination of the variable $X$ ($dom(f') = dom(f) - \{X\}$).

In discrete probability distributions, we marginalize out a variable by summing over all the outcomes of that variable.

### Exercise

Let's implement the marginalize function. We will provide most of the code, and you will fill in a few gaps.

In [ ]:
def marginalize(self, var):
    '''
    This function removes a variable from the domain, and sums over that variable in the table
    '''

    # create new domain
    new_dom = list(self.domain)
    new_dom.remove(var)

    # remove an axis of the table by summing it out
    axis = ... # TODO work out which axis to sum over
    new_table = ... # TODO sum over that axis

    # in the following line, `self.__class__` is the same as `Factor` (except it doesn't break things when subclassing)
    return self.__class__(tuple(new_dom),self.outcomeSpace, new_table)

Factor.marginalize = marginalize

#####################################
# Test code
#####################################

t_s_joint_prob = s_prob.join(t_prob)
f = t_s_joint_prob.marginalize('S')
print(f)

TypeError: 'ellipsis' object is not subscriptable

In [ ]:
# Answer

def marginalize(self, var):
    '''
    This function removes a variable from the domain, and sums over that variable in the table
    '''

    # create new domain
    new_dom = list(self.domain)
    new_dom.remove(var)

    # remove an axis of the table by summing it out
    axis = self.domain.index(var)
    new_table = np.sum(self.table, axis=axis)

    # in the following line, `self.__class__` is the same as `Factor` (except it doesn't break things when subclassing)
    return self.__class__(tuple(new_dom),self.outcomeSpace, new_table)

Factor.marginalize = marginalize

#####################################
# Test code
#####################################

t_s_joint_prob = s_prob.join(t_prob)
f = t_s_joint_prob.marginalize('S')
print(f)

╒══════╤══════╕
│ T    │   Pr │
╞══════╪══════╡
│ hot  │  0.5 │
├──────┼──────┤
│ cold │  0.5 │
╘══════╧══════╛



If you implemented the join operation correctly, you should see the following output:

```
╒══════╤══════╕
│ T    │   Pr │
╞══════╪══════╡
│ hot  │  0.5 │
├──────┼──────┤
│ cold │  0.5 │
╘══════╧══════╛

```

## Factor Normalization Operation

Factor normalization is useful when we make inference using evidence, since the resulting factor may not sum to one. To renormalize the factor to make it represent a probability distribution. Normalization is a simple, operation: we need to sum over all entries resulting in the value $Z$, and divide each factor entry by $Z$.

### Exercise

It is your turn. This time you will code the normalization function entirely. We have provided a stub for you.

In [ ]:
def normalize(self):
    '''
    Normalise the factor so that all probabilities add up to 1
    '''
    self.table = ... # TODO divide all elements of table by sum of every element
    return self

Factor.normalize = normalize

#####################################
# Test code
#####################################

w_prob_sun = w_prob.evidence(W='rain')

t_s_joint_prob = s_prob.join(t_prob)
t_s_w_joint_prob = t_s_joint_prob.join(w_prob_sun)
print(t_s_w_joint_prob.normalize())

TypeError: 'ellipsis' object is not subscriptable

In [ ]:
# Answer

def normalize(self):
    '''
    Normalise the factor so that all probabilities add up to 1
    '''
    self.table = self.table/np.sum(self.table)
    return self

Factor.normalize = normalize

#####################################
# Test code
#####################################

w_prob_sun = w_prob.evidence(W='rain')

t_s_joint_prob = s_prob.join(t_prob)
t_s_w_joint_prob = t_s_joint_prob.join(w_prob_sun)
print(t_s_w_joint_prob.normalize())

╒════════╤══════╤══════╤══════════╕
│ S      │ T    │ W    │       Pr │
╞════════╪══════╪══════╪══════════╡
│ summer │ hot  │ rain │ 0.141007 │
├────────┼──────┼──────┼──────────┤
│ summer │ cold │ rain │ 0.142446 │
├────────┼──────┼──────┼──────────┤
│ winter │ hot  │ rain │ 0.142446 │
├────────┼──────┼──────┼──────────┤
│ winter │ cold │ rain │ 0.574101 │
╘════════╧══════╧══════╧══════════╛



The expected output for the normalize function in the test case is the following:

```
╒════════╤══════╤══════╤══════════╕
│ S      │ T    │ W    │       Pr │
╞════════╪══════╪══════╪══════════╡
│ summer │ hot  │ rain │ 0.141007 │
├────────┼──────┼──────┼──────────┤
│ summer │ cold │ rain │ 0.142446 │
├────────┼──────┼──────┼──────────┤
│ winter │ hot  │ rain │ 0.142446 │
├────────┼──────┼──────┼──────────┤
│ winter │ cold │ rain │ 0.574101 │
╘════════╧══════╧══════╧══════════╛

```

# Entire factor class

Here is the full code for the Factor class, containing all the functions in this tutorial.

In [ ]:
from itertools import product
from tabulate import tabulate
import copy
import numpy as np
        
class Factor:
    '''
    Factors are a generalisation of discrete probability distributions over one or more random variables.
    Each variable must have a name (which may be a string or integer).
    The domain of the factor specifies which variables the factor operates over.
    The outcomeSpace specifies which 
    
    
    The probabilities are stored in a n-dimensional numpy array, using the domain and outcomeSpace
    as dimension and row labels respectively.
    '''
    def __init__(self, domain, outcomeSpace, table=None):
        '''
        Inititalise a factor with a given domain and outcomeSpace. 
        All probabilities are set to zero by default.
        '''
        self.domain = tuple(domain) # tuple of variable names, which may be strings, integers, etc.
        
        if table is None:
            # By default, intitialize with a uniform distribution
            self.table = np.ones(shape=tuple(len(outcomeSpace[var]) for var in self.domain))
            self.table = self.table/np.sum(self.table)
        else:
            self.table = table
            
        self.outcomeSpace = copy.copy(outcomeSpace)
    
    def __getitem__(self, outcomes):
        '''
        This function allows direct access to individual probabilities.
        E.g. if the factor represents a joint distribution over variables 'A','B','C','D', each with outcomeSpace [0,1,2],
        then `factor[0,1,0,2]` will return the probability that the four variables are set to 0,1,0,2 respectively.
        
        The outcomes used for indexing may also be strings. If in the example above the outcomeSpace of each variable is
        ['true','false'], then `factor['true','false','true','true']` will return the probability that 
        A='true', B='false', C='true' and D='true'. 
        
        The order of the indicies is determined by the order of the variables in self.domain.
        '''
        
        # check if only a single index was used.
        if not isinstance(outcomes, tuple):
            outcomes = (outcomes,)
            
        # convert outcomes into array indicies
        indicies = tuple(self.outcomeSpace[var].index(outcomes[i]) for i, var in enumerate(self.domain))
        return self.table[indicies]
    
    def __setitem__(self, outcomes, new_value):
        '''
        This function is called when setting a probability. E.g. `factor[0,1,0,2] = 0.5`.        
        '''
        if not isinstance(outcomes, tuple):
            outcomes = (outcomes,)
        indicies = tuple(self.outcomeSpace[var].index(outcomes[i]) for i, var in enumerate(self.domain))
        self.table[indicies] = new_value
            
    def join(self, other):
        '''
        This function multiplies two factors.
        '''
        # confirm that any shared variables have the same outcomeSpace
        for var in set(other.domain).intersection(set(self.domain)):
            if self.outcomeSpace[var] != other.outcomeSpace[var]:
                raise IndexError('Incompatible outcomeSpaces. Make sure you set the same evidence on all factors')

        # extend current domain with any new variables required
        new_dom = list(self.domain) + list(set(other.domain) - set(self.domain)) 
        
        # to prepare for multiplying arrays, we need to make sure both arrays have the correct number of axes
        self_t = self.table
        other_t = other.table
        for _ in set(other.domain) - set(self.domain):
            self_t = self_t[..., np.newaxis]     
        for _ in set(self.domain) - set(other.domain):
            other_t = other_t[..., np.newaxis]

        # And we need the new axes to be transposed to the correct location
        old_order = list(other.domain) + list(set(self.domain) - set(other.domain)) 
        new_order = []
        for v in new_dom:
            new_order.append(old_order.index(v))
        other_t = np.transpose(other_t, new_order)
        
        # Now that the arrays are all set up, we can rely on numpy broadcasting to work out which numbers need to be multiplied.
        # https://numpy.org/doc/stable/user/basics.broadcasting.html
        new_table = self_t * other_t
        
        # The final step is to create the new outcomeSpace
        new_outcomeSpace = self.outcomeSpace.copy()
        new_outcomeSpace.update(other.outcomeSpace)

        # in the following line, `self.__class__` is the same as `Factor` (except it doesn't break things when subclassing)
        return self.__class__(tuple(new_dom), new_outcomeSpace, table=new_table)
        
    def evidence(self, **kwargs):
        '''
        Sets evidence by modifying the outcomeSpace
        This function must be used to set evidence on all factors before joining,
        because it removes the relevant variable from the factor. 
        
        Usage: fac.evidence(A='true',B='false')
        This returns a factor which has set the variable 'A' to 'true' and 'B' to 'false'.
        '''
        f = self.copy()
        evidence_dict = kwargs
        for var, value in evidence_dict.items():
            if var in f.domain:
                
                # find the row index that corresponds to var = value
                index = f.outcomeSpace[var].index(value)
                
                # find the `var` axis and select only the row that corresponds to `value`
                # on all other axes, select every row
                slice_tuple = tuple(slice(index,index+1) if v == var else slice(None) for v in f.domain)
                f.table = f.table[slice_tuple]
                
                # modify the outcomeSpace to correspond to the changes just made to self.table
                f.outcomeSpace[var] = (value,)
        return f
    
    def marginalize(self, var):
        '''
        This function removes a variable from the domain, and sums over that variable in the table
        '''
        
        # create new domain
        new_dom = list(self.domain)
        new_dom.remove(var) 
        
        # remove an axis of the table by summing it out
        axis = self.domain.index(var)
        new_table = np.sum(self.table, axis=axis)
        
        # in the following line, `self.__class__` is the same as `Factor` (except it doesn't break things when subclassing)
        return self.__class__(tuple(new_dom),self.outcomeSpace, new_table)
    
    def copy(self):
        return copy.deepcopy(self)
    
    def normalize(self):
        '''
        Normalise the factor so that all probabilities add up to 1
        '''
        self.table = self.table/np.sum(self.table)
        return self
    
    def __mul__(self, other):
        '''
        Override the * operator, so that it can be used to join factors
        '''
        return self.join(other)
            
    def __str__(self):
        '''
        This function determines the string representation of this object.
        In this case, the function prints out a row for every possible instantiation 
        of the factor, along with the associated probability.
        This function will be called whenever you print out this object, i.e.
        a_prob = Factor(...)
        print(a_prob)
        '''
        table = []
        outcomeSpaces = [self.outcomeSpace[var] for var in self.domain]
        for key in product(*outcomeSpaces):
            row = list(key)
            row.append(self[key])
            table.append(row)
        header = list(self.domain) + ['Pr']
        return tabulate(table, headers=header, tablefmt='fancy_grid') + '\n'


# Reinitialize these objects using the final version of the class 
s_prob = Factor(('S',), outcomeSpace)
s_prob['summer'] = 0.5
s_prob['winter'] = 0.5

t_prob = Factor(('S','T'), outcomeSpace)
t_prob['summer','hot']  = 0.7
t_prob['summer','cold'] = 0.3
t_prob['winter','hot']  = 0.3
t_prob['winter','cold'] = 0.7

w_prob = Factor(('S','T','W'), outcomeSpace)
w_prob['summer','hot','sun']   = 0.86
w_prob['summer','hot','rain']  = 0.14
w_prob['summer','cold','sun']  = 0.67
w_prob['summer','cold','rain'] = 0.33
w_prob['winter','hot','sun']   = 0.67
w_prob['winter','hot','rain']  = 0.33
w_prob['winter','cold','sun']  = 0.43
w_prob['winter','cold','rain'] = 0.57

We have reached the end of this tutorial. Now we have all the tools we need to start making inference on Graphical Models. Also, you can use this code to check the results of your calculations in the theory part of this tutorial. For instance, in **Question 6**, we asked for the joint probability table $P(T,S,W)$. We can calculate this with a single line of python code.

In [ ]:
print(t_prob.join(s_prob).join(w_prob))

╒════════╤══════╤══════╤════════╕
│ S      │ T    │ W    │     Pr │
╞════════╪══════╪══════╪════════╡
│ summer │ hot  │ sun  │ 0.301  │
├────────┼──────┼──────┼────────┤
│ summer │ hot  │ rain │ 0.049  │
├────────┼──────┼──────┼────────┤
│ summer │ cold │ sun  │ 0.1005 │
├────────┼──────┼──────┼────────┤
│ summer │ cold │ rain │ 0.0495 │
├────────┼──────┼──────┼────────┤
│ winter │ hot  │ sun  │ 0.1005 │
├────────┼──────┼──────┼────────┤
│ winter │ hot  │ rain │ 0.0495 │
├────────┼──────┼──────┼────────┤
│ winter │ cold │ sun  │ 0.1505 │
├────────┼──────┼──────┼────────┤
│ winter │ cold │ rain │ 0.1995 │
╘════════╧══════╧══════╧════════╛



Since we overrode the * operater using the `__mul__` function above, we can also do:

In [ ]:
print(t_prob*s_prob*w_prob)

╒════════╤══════╤══════╤════════╕
│ S      │ T    │ W    │     Pr │
╞════════╪══════╪══════╪════════╡
│ summer │ hot  │ sun  │ 0.301  │
├────────┼──────┼──────┼────────┤
│ summer │ hot  │ rain │ 0.049  │
├────────┼──────┼──────┼────────┤
│ summer │ cold │ sun  │ 0.1005 │
├────────┼──────┼──────┼────────┤
│ summer │ cold │ rain │ 0.0495 │
├────────┼──────┼──────┼────────┤
│ winter │ hot  │ sun  │ 0.1005 │
├────────┼──────┼──────┼────────┤
│ winter │ hot  │ rain │ 0.0495 │
├────────┼──────┼──────┼────────┤
│ winter │ cold │ sun  │ 0.1505 │
├────────┼──────┼──────┼────────┤
│ winter │ cold │ rain │ 0.1995 │
╘════════╧══════╧══════╧════════╛



## Final Task

Create a python file called `DiscreteFactors.py` containing the `Factor` class, and save it to your computer. It will be used in future tutorials.